In [63]:
import os
import json
import cohere
import re
import logging
from datetime import datetime

from langchain_openai import AzureOpenAIEmbeddings
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType
from azure.search.documents._generated.models import QueryCaptionResult

from openai import AzureOpenAI
import tiktoken

from dotenv import load_dotenv

load_dotenv()

def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

class AzureEmbeddings:

    def __init__(self):
        pass

    @staticmethod
    def get_embedding():
        return AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("OPENAI_AZURE_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            azure_endpoint=os.getenv("OPEN_AI_AZURE_URL")
        )

    @staticmethod
    def generate_embeddings(content: str):
        embeddings = AzureOpenAIEmbeddings(
            azure_deployment=os.getenv("OPENAI_AZURE_DEPLOYMENT"), 
            openai_api_version="2023-08-01-preview",
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            azure_endpoint=os.getenv("OPEN_AI_AZURE_URL")
        )

        doc_result = embeddings.embed_documents([content])

        return doc_result[0]

openai_client = AzureOpenAI(
    api_key=os.getenv("AZURE_CHAT_OPENAI_API_KEY"), 
    api_version=os.getenv("AZURE_CHAT_OPENAI_API_VERSION"), 
    azure_endpoint=os.getenv("AZURE_CHAT_OPENAI_ENDPOINT")
    )

embeddings_client = AzureEmbeddings()
store_search_url: str = f'https://{os.getenv('AZURE_COGNITIVE_SEARCH_SERVICE_NAME')}.search.windows.net'
search_client = SearchClient(
            store_search_url, os.getenv("AZURE_COGNITIVE_SEARCH_INDEX_NAME"),
            AzureKeyCredential(os.getenv("AZURE_COGNITIVE_SEARCH_API_KEY"))
        )

co = cohere.Client(api_key=os.getenv('COHERE_API_KEY'))

In [64]:
log_file_name = ""


def ensure_directory(directory_name):
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)


def log(message):
    global log_file_name
    ensure_directory("logs")
    with open(log_file_name, "a") as file:
        file.write(f"{datetime.now()}\n{message}\n")

In [65]:
conversation_price = 0
conversation_ct = 0
conversation_pt = 0
conversation_tt = 0


def get_conversation_price(new_completion):
    global conversation_price, conversation_ct, conversation_pt, conversation_tt
    USD_price_in_COP = 4100

    # gpt4o
    ct_price = 0.000015
    pt_price = 0.000005

    ct = new_completion.usage.completion_tokens
    pt = new_completion.usage.prompt_tokens
    tt = new_completion.usage.total_tokens
    usd_total_price = (ct * ct_price) + (pt * pt_price)
    cop_total_price = usd_total_price * USD_price_in_COP
    conversation_ct += ct
    conversation_pt += pt
    conversation_tt += tt
    conversation_usd_total_price = (conversation_ct * ct_price) + (
        conversation_pt * pt_price
    )
    conversation_cop_total_price = conversation_usd_total_price * USD_price_in_COP

    print(
        f"💰 Conversation price: ${conversation_cop_total_price} COP (In: {conversation_pt}, Out: {conversation_ct}, Total token: {conversation_tt})\n"
        f"   This message: ${cop_total_price} COP (In: {pt}, Out: {ct}, Total token: {tt}) "
        f""
    )
    log(
        f"💰 Conversation price: ${conversation_cop_total_price} COP (In: {conversation_pt}, Out: {conversation_ct}, Total token: {conversation_tt})\n "
        f"   This message: ${cop_total_price} COP (In: {pt}, Out: {ct}, Total token: {tt}) "
        f""
    )

    return

In [66]:
### 🔑 Definir system prompt y tools

messages = []
user_input = ""

query_prompt = "Utiliza esta herramienta si necesitas más fuentes para responder al usuario con precisión. Genera dos consultas de búsqueda basadas en lo que el usuario te ha preguntado: una consulta de texto completo utilizando sintaxis Lucene y una consulta semántica. Cada una debe contener la idea completa y fuentes específicas si son necesarias. Si la pregunta del usuario aborda cuándo, cómo o qué información, debes incluir eso en las consultas, ya que afectará los resultados. Evitar hacer búsquedas de coincidencias exactas de palabras, ya que es probables que los textos esten mal transcritos. Utiliza búsquedas por campo, especialmente siempre busca en el campo 'content' y utiliza los campos 'title' y 'author' si son útiles para obtener el fragmento correcto del documento. La consulta semántica debe ser más general y capturar el significado completo de lo que el usuario está preguntando. Si necesitas diferentes fuentes, busca cada una con un llamado a esta tool"

search_legal_info = {
    "type": "function",
    "function": {
        "name": "search_legal_info",
        "description": query_prompt,
        "parameters": {
            "type": "object",
            "strict": "true",
            "properties": {
                "full_text_query": {
                    "type": "string",
                    "description": "La consulta de texto completo utilizando sintaxis Lucene. Debe enfocarse en los campos 'content', 'title' y 'author' para encontrar los fragmentos correctos, evitando depender de coincidencias exactas de palabras debido a posibles errores de transcripción en los documentos. Ejemplo: 'content:causales de agravación artículo 267 AND title:ley 599 del 2000'",
                },
                "semantic_query": {
                    "type": "string",
                    "description": "La consulta semántica que captura el significado completo de lo que el usuario está preguntando. Debe ser más general y no utilizar sintaxis específica de campos o operadores lógicos. Ejemplo: 'causales de agravación de la estafa artículo 267 ley 599 del 2000'",
                },
            },
            "required": ["semantic_query", "full_text_query"],
        },
    },
}
get_next_chunk = {
    "type": "function",
    "function": {
        "name": "get_next_chunk",
        "description": "Use this tool to get when an important sources comes incomplete or the content is cut off. This tool will help you retrieve the following section of that given source.",
        "parameters": {
            "type": "object",
            "properties": {
                "source_id": {
                    "type": "string",
                    "description": "the unique identifier of the chunk that is incomplete. eg:'20240719192458csjscpboletinjurisprudencial20181219pdf_chunk30'",
                },
            },
            "required": ["source_id"],
        },
    },
}

response_system_template = f'Eres Ariel, un asistente para la investigación legal en jurisprudencia colombiana. Sigue estas instrucciones al responder:\n\nConsulta de Fuentes:\n\nUtiliza la herramienta "search_legal_info" para buscar información relevante.\nCrea consultas específicas que incluyan toda la información necesaria.\nSi es necesario, genera múltiples consultas para obtener diferentes tipos de documentos.\nRedacción de Respuestas:\n\nSé detallado y preciso, utilizando exclusivamente la información de las fuentes proporcionadas.\nNo incluyas información que no haya sido extraída directamente de las fuentes.\nCitación de Fuentes:\n\nCita el mayor número de fuentes aplicables.\nUtiliza corchetes para referenciar la fuente con el ID del fragmento sin modificarlo, por ejemplo: [12345_id_de_ejemplo].\nNo combines fuentes; lista cada fuente por separado.\nFormato de Respuesta:\n\nEscribe tus respuestas en formato HTML, sin incluir los tags "```html" al principio o al final.\nUtiliza etiquetas HTML estándar como <p>, <strong>, <em>, etc.\nManejo de Fragmentos Incompletos:\n\nSi un fragmento es necesario pero está incompleto, utiliza "get_next_chunk" con el source_id correspondiente para obtener el siguiente fragmento.\nInterpretación de Categorías:\n\nPrioriza las fuentes según su categoría:\nConstitucional\nLegal\nInfralegal\nJurisprudencia\nDoctrina\nDa mayor importancia a los fragmentos más actuales.\nPrecisión Legal:\n\nSi un extracto cita otra fuente (artículo, ley, etc.), búscala directamente para obtener información de primera mano.\nNo confundas leyes o artículos; si el usuario pregunta por una ley específica, ignora extractos de otras leyes.\nConsultas sin Resultados:\n\nSi las fuentes no ayudan a responder la pregunta, responde: "No encuentro información con esos términos, ¿puedes reformular tu consulta?"'

# query_system_template = f"""Eres un experto legal que necesita buscar en una base de conocimiento con cientos de miles de documentos, fuentes confiables y precisas para ayudar a responder las consultas del usuario. He traído algunos extractos de documentos que pueden ser útiles para que identifiques fuentes que puedes incluir en tu término de búsqueda si ves que son aplicables. La base de conocimientos está alojada en Azure AI Search y bebes generar un query en formato JSON para buscar allí.
# Los documentos están guardados en fragmentos con los siguientes campos:
# - id: Identificador único del fragmento
# - title: Título del documento
# - author: Autor del documento
# - keywords: Tema legal, puede ser: General, Constitucional, Internacional_Publico, Internacional_Privado, Penal, Financiero, entre otros.
# - category: Tipo de documento legal. Las opciones son: Jurisprudencia, Doctrina, Constitucional, Legal, Infralegal y Otros_temas_legales.
# - page: Página de la que fue extraído el fragmento.
# - year: Año en el que se publicó el documento.
# - content: 500 tokens de contenido, fragmento o extracto del documento. Son solo 500 tokens del documento completo.
# El JSON que debes generar contiene:
# 1. text_query =  El término de búsqueda escrito en sintaxis Lucena. Debe enfocarse en los campos "content", "title" y "author" para encontrar los fragmentos correctos, evitando depender de coincidencias exactas de palabras debido a posibles errores de transcripción en los documentos.
# 2. semantic_query = El término de búsqueda semántico que captura el significado completo de lo que el usuario está preguntando. Debe ser más general y no utilizar sintaxis específica de campos o operadores lógicos.
# 3. category_weight = El peso en importancia de las categorías. En la gran mayoría de casos, los pesos los distribuirás de la siguiente manera "category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2"
# 4. filters = Este campo es opcional y solo si lo ves extrictamente necesario. Este campo va a afectar mucho tu búsqueda, así que solo úsalo si estás 100% convencido que lo necesitas. Aquí podrás reducir tu búsqueda manipulando los filtros del query utilizando la sintazis $filter OData de Azure AI Search, ten en cuenta los campos y los valores utilizados para guardar los fragmentos a la hora de generar un filtro.
# Indicaciones para los términos de búsqueda:
# - Cada uno debe contener la idea completa y fuentes específicas si son necesarias. Si la pregunta del usuario aborda cuándo, cómo o qué información, debes incluir eso en las consultas, ya que afectará los resultados.
# - Si hay varias formas de referise a una fuente, incluye los diferentes nombres de esa fuente en los términos de búsqueda.
# - Si necesitas buscar diferentes fuentes, crea otro JSON con la búsqueda que necesitas.
# - Sé conciso y no te incluyas palabras de más. Los fragmentos de referencias no deben desviarte de la búqueda inicial del usaurio.
# """
query_system_template = f"""Eres un experto legal que necesita buscar en una base de conocimiento con cientos de miles de documentos, fuentes confiables y precisas para ayudar a responder las consultas del usuario. He traído algunos extractos de documentos que pueden ser útiles para que identifiques fuentes que puedes incluir en tu término de búsqueda si ves que son aplicables. La base de conocimientos está alojada en Azure AI Search y bebes generar un query en formato JSON para buscar allí. 
Los documentos están guardados en fragmentos con los siguientes campos:
- id: Identificador único del fragmento
- title: Título del documento
- author: Autor del documento
- keywords: Tema legal, puede ser: General, Constitucional, Internacional_Publico, Internacional_Privado, Penal, Financiero, entre otros.
- category: Tipo de documento legal. Las opciones son: Jurisprudencia, Doctrina, Constitucional, Legal, Infralegal y Otros_temas_legales. 
- page: Página de la que fue extraído el fragmento.
- year: Año en el que se publicó el documento.
- content: 500 tokens de contenido, fragmento o extracto del documento. Son solo 500 tokens del documento completo.
Requisitos para generar el términos de búsqueda:
- Debes generar un query que sirva para ejecutar una búsqueda semántica e híbrida.
- El query debe contener la idea completa y fuentes específicas si son necesarias pero no ser demasiado larga. Sé conciso.
- Si la pregunta del usuario aborda cuándo, cómo o qué información, debes incluir eso en las consultas, ya que afectará los resultados.
- Si hay varias formas de referise a una fuente, incluye los diferentes nombres de esa fuente en el query.
- Si necesitas buscar diferentes fuentes, crea otro JSON con la búsqueda que necesitas.
- Los fragmentos de referencias no deben desviarte de la búqueda inicial del usaurio.
"""

query_few_shots = [
    {"role": "user", "content": "¿Qué dice el artículo 103 del Código Penal?"},
    {
        "role": "assistant",
        "content": '{"text_query":"content:homicidio artículo 103 AND title:"ley 599 del 2000" OR "código penal"","semantic_query":"homicidio artículo 103 ley 599 del 2000 código penal","category_weight":{"Constitucional":6,"Legal":5,"Infralegal":4,"Jurisprudencia":3,"Doctrina":2,"Otros_temas_legales":1},"filters":""}',
    },
    {
        "role": "user",
        "content": "¿Cuáles son las causales genéricas de atenuación punitiva?",
    },
    {
        "role": "assistant",
        "content": '{"text_query":"content:causales genéricas de atenuación punitiva AND title:ley 599 del 2000","semantic_query":"causales genéricas de atenuación punitiva ley 599 del 2000 código penal colombiano","category_weight":{"Constitucional":6,"Legal":5,"Infralegal":4,"Jurisprudencia":3,"Doctrina":2,"Otros_temas_legales":1},"filters":""}',
    },
]

In [67]:
def format_search_results(docs_list):
    print(docs_list)
    documents = []
    try:
        for document in docs_list:
            print(document)
            captions: QueryCaptionResult = document.get("@search.captions", "")
            captions_text = " // ".join([caption.text for caption in captions]) if captions is not None else ""
            doc_formatted = {
                "position" : len(documents) + 1,
                "score": document.get("@search.score", 10),
                "rerank": document.get("@search.reranker_score", 10),
                "captions": captions_text,
                "id": document["id"],
                "title": document["title"],
                "author": document["author"],
                "keywords": document["keywords"],
                "category": document["category"],
                "page": document["page"],
                "year": document["year"],
                "has_copyright": document["has_copyright"],
                "file_path": document["file_path"],
                "external_id": document["external_id"],
                "content": document["content"],
            }
            documents.append(doc_formatted)
        log(
            f">>>>>>>> format_search_results:\n"
            f"{json.dumps(documents, indent=4)}"
            )
    except Exception as e:
        print(e)
    print("Sucesssss")
    return documents

def format_search_results_support(docs_list):
    print(docs_list)
    documents = []
    try:
        for document in docs_list:
            print(document)
            doc_formatted = {
                "content": document.get("content", "none"),
            }
            documents.append(doc_formatted)
        log(
            f">>>>>>>> format_search_results:\n"
            f"{json.dumps(documents, indent=4)}"
            )
    except Exception as e:
        print(e)
    print("Sucesssss")
    return documents

def format_reranked_results(docs_list):
    documents = []
    for document in docs_list:
        doc_formatted = {
            "position" : len(documents) + 1,
            "id": document.document.id,
            "relevance": document.relevance_score,
            "score": document.document.score,
            "rerank": document.document.rerank,
            "captions": document.document.captions,
            "title": document.document.title,
            "author": document.document.author,
            "keywords": document.document.keywords,
            "category": document.document.category,
            "page": document.document.page,
            "year": document.document.year,
            "has_copyright": document.document.has_copyright,
            "file_path": document.document.file_path,
            "external_id": document.document.external_id,
            "content": document.document.content,
        }
        documents.append(doc_formatted)
    log(
        f">>>>>>> format_reranked_results:\n"
        f"{json.dumps(documents, indent=4)}"        
    )
    return documents

def filter_docs(doc_list):
    filtered_docs = []
    contents = [doc["content"] for doc in doc_list]
    
    for doc in doc_list:
        score = doc.get("relevance", 10) 
        rerank = doc.get("relevance", 10) 
        relevance = doc.get("relevance", 1)
        if (
            contents.count(doc["content"]) == 1
            and (relevance > 0.65
            and rerank > 2
            or score > 0.025)
        ):
            filtered_docs.append(doc)
    print(f">> 🧹 Docs filtrados. De {len(doc_list)} pasaron {len(filtered_docs)} ...")
    log(f">> 🧹 Docs filtrados. De {len(doc_list)} pasaron {len(filtered_docs)} ...\n"
    f"{json.dumps(filtered_docs, indent=4)}")
    return filtered_docs

def filter_docs_sources(doc_list):
    filtered_docs = []
    contents = [doc["content"] for doc in doc_list]
    
    for doc in doc_list:
        if (
            contents.count(doc["content"]) == 1
        ):
            filtered_docs.append(doc)
    print(f">> 🧹 Docs filtrados. De {len(doc_list)} pasaron {len(filtered_docs)} ...")
    # log(f">> 🧹 Docs filtrados. De {len(doc_list)} pasaron {len(filtered_docs)} ...\n"
    # f"{json.dumps(filtered_docs, indent=4)}")
    return filtered_docs

def add_doc_to_context(docs_list):
    sources = ""
    counter = 0
    
    for document in docs_list:
        counter += 1
        sources += (
            f"Fuente #{counter}\n"
            f"id: {document["id"]}\n"
            f"title: {document["title"]}\n"
            f"author: {document["author"]}\n"
            f"year: {document["year"]}\n"
            f"keywords: {document["keywords"]}\n"
            f"category: {document["category"]}\n"
            f"page: {document["page"]}\n"
            f"content: {document["content"]}\n"
            f"\n\n"
        )      
    log(
        f"To add in Context: \n"
        f"{json.dumps({"sources":sources}, indent=4)}"
        )
    return sources
    

In [68]:
def search_for_chunks(text_query, vector_query, rerank_query):
    log(
        f">> 🔎 Buscando documentos Text Query: {text_query} ...\n"
        f"      Buscando documentos Semantic Query: {rerank_query} ...\n"
    )
    print(
        f">> 🔎 Buscando documentos Text Query: {text_query} ...\n"
        f">>    Buscando documentos Semantic Query: {rerank_query} ...\n"
    )
    results_num = 50
    results = search_client.search(
        search_text=text_query,
        vector_queries=[
            {
                "vector": vector_query,
                "k": results_num,
                "fields": "content_vector",
                "kind": "vector",
                "exhaustive": True,
            }
        ],
        top=results_num,
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name="default",
        query_caption="extractive|highlight-false",
        scoring_profile="legal2",
        scoring_parameters=[
            "categoryBoost-Constitucional,Legal,Infralegal,Jurisprudencia,Doctrina"
        ],
    )

    # Convertir en Lucene full text syntax y pedir otro query semántico
    #

    results_formatted = format_search_results(results)
    restults_filtered = filter_docs(results_formatted)

    log(f">> ⭐️ Aplicando reorganización semántica: {rerank_query} ...")
    print(f">> ⭐️ Aplicando reorganización semántica ...")
    reranked_docs = co.rerank(
        model="rerank-multilingual-v3.0",
        top_n=10,
        query=rerank_query,
        documents=restults_filtered,
        return_documents=True,
        rank_fields=["content", "title", "author", "keywords", "category"],
    )
    reranked_docs_formatted = format_reranked_results(reranked_docs.results)
    reranked_docs_filtered = filter_docs(reranked_docs_formatted)

    # reranked_docs_filtered = restults_filtered[:10]

    return reranked_docs_filtered


def search_semantic(query):
    # log(
    #     f">> 🔎 Buscando documentos Text Query: {text_query} ...\n"
    #     f"      Buscando documentos Semantic Query: {rerank_query} ...\n"
    # )
    print(f">> 🔎 Buscando documentos de soporte: {query} ...\n")

    vector_query = embeddings_client.generate_embeddings(content=query)

    results_num = 20
    results = search_client.search(
        filter="category eq 'Jurisprudencia' or category eq 'Doctrina'",
        vector_queries=[
            {
                "vector": vector_query,
                "k": results_num,
                "fields": "content_vector",
                "kind": "vector",
                "exhaustive": True,
            }
        ],
        top=results_num,
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name="default",
        query_caption="extractive|highlight-false",
    )
    results_formatted = format_search_results_support(results)
    restults_filtered = filter_docs_sources(results_formatted)

    return restults_filtered[:3]


def get_next_chunks(id):
    log(f">> ⏩️ Buscando los chunks siguientes de {id} ...")
    print(f">> ⏩️ Buscando los chunks siguientes de {id} ...")
    pattern = r"^(.*_chunk)(\d+)$"
    match = re.search(pattern, id)

    if match:
        prefix = match.group(1)
        chunk_number = int(match.group(2))
        next_chunks_result = search_client.search(
            filter=f"id eq '{prefix}{chunk_number + 1}' or id eq '{prefix}{chunk_number + 2}'",
            top=2,
        )
        return format_search_results(list(next_chunks_result))


def get_next_chunk_tool(source_id):
    next_chunks = get_next_chunks(source_id)
    log(f"Continuación de {source_id}:\n{add_doc_to_context(next_chunks)}")
    return f"Continuación de {source_id}:\n{add_doc_to_context(next_chunks)}"


def search_legal_info_tool(text_query, semantic_query):
    category_weight = " AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2"

    vector_query = embeddings_client.generate_embeddings(content=semantic_query)

    results = search_for_chunks(
        text_query=text_query,
        rerank_query=semantic_query,
        vector_query=vector_query,
    )
    return f"Extractos de documentos encontrados: \n{add_doc_to_context(results)}"

In [69]:
def generate_completion():
    global messages
    log(f"{json.dumps(messages, indent=4)}\n")
    response = openai_client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
        messages=messages,
        tools=[get_next_chunk, search_legal_info],
        temperature=0.2,
        n=1,
        tool_choice="auto",
    )
    log(response)
    get_conversation_price(response)
    return response


from pydantic import BaseModel


class Categories(BaseModel):
    Constitucional: int
    Legal: int
    Infralegal: int
    Jurisprudencia: int
    Doctrina: int
    Otros_temas_legales: int


class SearchQuery(BaseModel):
    text_query: str
    semantic_query: str
    category_weight: Categories
    filters: str


schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "search_query",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "text_query": {
                    "type": "string",
                    "description": "El término de búsqueda escrito en sintaxis Lucena. Debe enfocarse en los campos 'content', 'title' y 'author' para encontrar los fragmentos correctos, evitando depender de coincidencias exactas de palabras debido a posibles errores de transcripción en los documentos. Utiliza 'content:'seguido del query que crees que puedes encontrar dentro del fragmento escrito del documento. Utiliza 'title:'seguido de como crees que el documento se llama",
                },
                "semantic_query": {
                    "type": "string",
                    "description": "El término de búsqueda semántico que captura el significado completo de lo que el usuario está preguntando. Debe ser más general y no utilizar sintaxis específica de campos o operadores lógicos.",
                },
                "category_weight": {
                    "type": "object",
                    "properties": {
                        "Constitucional": {
                            "type": "integer",
                            "description": "Weight for the Constitucional category. Defaults to 6 in most cases unless specified by the user question",
                        },
                        "Legal": {
                            "type": "integer",
                            "description": "Weight for the Legal category. Defaults to 5 in most cases unless specified by the user question",
                        },
                        "Infralegal": {
                            "type": "integer",
                            "description": "Weight for the Infralegal category. Defaults to 4 in most cases unless specified by the user question",
                        },
                        "Jurisprudencia": {
                            "type": "integer",
                            "description": "Weight for the Jurisprudencia category. Defaults to 3 in most cases unless specified by the user question",
                        },
                        "Doctrina": {
                            "type": "integer",
                            "description": "Weight for the Doctrina category. Defaults to 2 in most cases unless specified by the user question",
                        },
                        "Otros_temas_legales": {
                            "type": "integer",
                            "description": "Weight for other legal topics. Defaults to 1 in most cases unless specified by the user question",
                        },
                    },
                    "required": [
                        "Constitucional",
                        "Legal",
                        "Infralegal",
                        "Jurisprudencia",
                        "Doctrina",
                        "Otros_temas_legales",
                    ],
                    "additionalProperties": False,
                    "description": "El peso en importancia de las categorías. En la gran mayoría de casos, los pesos los distribuirás de la siguiente manera 'category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2'",
                },
                "filters": {
                    "type": "string",
                    "description": "Este campo es opcional y solo si lo ves extrictamente necesario. Este campo va a afectar mucho tu búsqueda, así que solo úsalo si estás 100% convencido que lo necesitas. Aquí podrás reducir tu búsqueda manipulando los filtros del query utilizando la sintazis $filter OData de Azure AI Search, ten en cuenta los campos y los valores utilizados para guardar los fragmentos a la hora de generar un filtro.",
                },
            },
            "required": ["text_query", "semantic_query", "category_weight", "filters"],
            "additionalProperties": False,
        },
    },
}


def generate_queries():
    global messages
    # log(f"{json.dumps(messages, indent=4)}\n")
    response = openai_client.chat.completions.create(
        model=os.getenv("AZURE_CHAT_OPENAI_DEPLOYMENT"),
        messages=messages,
        response_format=schema,
        temperature=0.0,
    )
    log(response)
    get_conversation_price(response)
    return response

In [70]:
def append_message(new_message):
    global messages
    conversation = messages

    if isinstance(new_message, dict):
        role = new_message.get("role")
        content = new_message.get("content")
    else:
        role = new_message.role
        content = new_message.content

    if role == "assistant":
        if new_message.tool_calls:
            tool_calls = new_message.tool_calls
            tool_calls_formatted = []
            for tool_call in tool_calls:
                tool_calls_formatted.append(
                    {
                        "id": tool_call.id,
                        "function": {
                            "arguments": str(json.loads(tool_call.function.arguments)),
                            "name": tool_call.function.name,
                        },
                        "type": "function",
                    }
                )
            conversation.append(
                {
                    "role": "assistant",
                    "tool_calls": tool_calls_formatted,
                },
            )
            return conversation  ## Assistant call tools
        elif content:
            conversation.append({"role": "assistant", "content": content})
            return conversation  ## Assistant talk
    elif role == "tool":
        conversation.append(new_message)
        return conversation  ## Tool reponse
    else:
        return conversation

In [71]:
def call_tools(tool_calls):
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        tool_args = json.loads(tool_call.function.arguments)
        if tool_name == "get_next_chunk":
            content = get_next_chunk_tool(tool_args.get("source_id"))
        elif tool_name == "search_legal_info":
            content = search_legal_info_tool(
                text_query=tool_args.get("full_text_query"),
                semantic_query=tool_args.get("semantic_query"),
            )

        new_tool_message = {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": tool_name,
            "content": content,
        }

        append_message(new_message=new_tool_message)

    print(">> 🤖 Generando respuesta ...")
    new_assistant_completion = generate_completion()
    new_assistant_message = new_assistant_completion.choices[0].message
    append_message(new_message=new_assistant_message)

    if new_assistant_message.content:
        return print("💬 Assistant:" + new_assistant_message.content)
    else:
        return call_tools(
            tool_calls=new_assistant_message.tool_calls,
        )

In [72]:
def run_conversation(user_prompt=None):
    if user_prompt:
        get_answer(user_prompt)
    else:
        while True:
            user_input = input("You: ")
            if user_input.lower() == "exit":
                print("Exiting chat...")
                break
            get_answer(user_input)


def get_answer(user_prompt):
    global messages, log_file_name, user_input

    user_input = user_prompt

    current_time = datetime.now().strftime("%m-%d %H:%M")
    log_file_name = f"logs/{current_time} - {user_input[:50]}.log"

    print("🙎‍♂️ User: " + user_input)
    log(f"User input: {user_input}")
    new_user_message = {"role": "user", "content": user_input}

    messages = [
        system_message,
        new_user_message,
    ]

    print(">> 🤖 Generando respuesta ...")
    response = generate_completion()
    response_message = response.choices[0].message

    messages = append_message(new_message=response_message)

    tool_calls = response_message.tool_calls
    if tool_calls:
        call_tools(tool_calls=tool_calls)
    elif response_message.content:
        print("💬 Assistant:", response_message.content)
    else:
        print("An error ocurred during completion generation - response: ", response)

In [89]:
message = []
run_conversation("¿Cuáles son las causales genéricas de atenuación punitiva?")

🙎‍♂️ User: ¿Cuáles son las causales genéricas de atenuación punitiva?
>> 🤖 Generando respuesta ...
💰 Conversation price: $26.096500000000002 COP (In: 865, Out: 136, Total token: 1001)
   This message: $26.096500000000002 COP (In: 865, Out: 136, Total token: 1001) 
>> 🔎 Buscando documentos Text Query: content:causales genéricas de atenuación punitiva AND title:ley 599 del 2000 AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2 ...
>> 🔎 Buscando documentos Semantic Query: causales genéricas de atenuación punitiva ley 599 del 2000 ...

>> 🧹 Docs filtrados. De 50 pasaron 16 ...
>> ⭐️ Aplicando reorganización semántica ...
>> 🧹 Docs filtrados. De 10 pasaron 10 ...
>> 🔎 Buscando documentos Text Query: content:causales genéricas de atenuación punitiva AND title:código penal colombiano AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2 ...
>> 🔎 Buscando documentos Semantic Query: causales genéricas de atenuación punitiva código penal colo

In [ ]:
messages = []
run_conversation("¿Qué dice el artículo 103 del código penal?")

🙎‍♂️ User: ¿Qué dice el artículo 103 del código penal?
>> 🤖 Generando respuesta ...
💰 Conversation price: $268.93950000000007 COP (In: 10803, Out: 772, Total token: 11575)
   This message: $21.443 COP (In: 923, Out: 41, Total token: 964) 
>> 🔎 Buscando documentos: artículo 103 ley 599 del 2000 código penal ...
>> 🧹 Docs filtrados. De 50 pasaron 30 ...
>> ⭐️ Aplicando reorganización semántica ...
>> 🧹 Docs filtrados. De 10 pasaron 7 ...
>> 🤖 Generando respuesta ...
💰 Conversation price: $363.54699999999997 COP (In: 15334, Out: 800, Total token: 16134)
   This message: $94.6075 COP (In: 4531, Out: 28, Total token: 4559) 
>> ⏩️ Buscando los chunks siguientes de 20240814181609ley599de2000pdf_chunk69 ...
>> 🤖 Generando respuesta ...
💰 Conversation price: $483.3695 COP (In: 20918, Out: 887, Total token: 21805)
   This message: $119.82250000000002 COP (In: 5584, Out: 87, Total token: 5671) 
💬 Assistant:El artículo 103 del Código Penal colombiano, contenido en la Ley 599 de 2000, establece lo 

In [ ]:
# messages = []
# run_conversation(
#     "Daniel Andrés Fúquenes Barriga, en su condición de auxiliar de la justicia y secuestre, recaudó una suma de dinero por concepto de arrendamiento, específicamente $684,000 entre febrero y julio de 2015. Este dinero debía ser entregado a su dueño o poseedor, pero Fúquenes Barriga retuvo la suma para sí mismo, incumpliendo con la obligación de devolverla. Este acto de retención y apropiación del dinero, que se le había confiado por un título no traslativo de dominio, ¿Qué delito cometió?"
# )

In [ ]:
messages = []
run_conversation("Dame el artículo 55 del código penal")

🙎‍♂️ User: Dame el artículo 55 del código penal
>> 🤖 Generando respuesta ...
💰 Conversation price: $187.88250000000002 COP (In: 7854, Out: 437, Total token: 8291)
   This message: $20.992 COP (In: 922, Out: 34, Total token: 956) 
>> 🔎 Buscando documentos: artículo 55 código penal ...
>> 🧹 Docs filtrados. De 50 pasaron 14 ...
>> ⭐️ Aplicando reorganización semántica ...
>> 🧹 Docs filtrados. De 10 pasaron 0 ...
>> 🤖 Generando respuesta ...
💰 Conversation price: $210.08400000000003 COP (In: 8832, Out: 472, Total token: 9304)
   This message: $22.201500000000003 COP (In: 978, Out: 35, Total token: 1013) 
>> ⏩️ Buscando los chunks siguientes de 20240719192458csjscpboletinjurisprudencial20181219pdf_chunk30 ...
>> 🤖 Generando respuesta ...
💰 Conversation price: $247.49650000000003 COP (In: 9880, Out: 731, Total token: 10611)
   This message: $37.4125 COP (In: 1048, Out: 259, Total token: 1307) 
💬 Assistant:El artículo 55 del Código Penal colombiano establece lo siguiente:

**Artículo 55. Circ

In [102]:
messages = []
run_conversation("¿Qué se entiende por “documento” en el derecho penal?")

🙎‍♂️ User: ¿Qué se entiende por “documento” en el derecho penal?
>> 🤖 Generando respuesta ...
💰 Conversation price: $24.2515 COP (In: 862, Out: 107, Total token: 969)
   This message: $24.2515 COP (In: 862, Out: 107, Total token: 969) 
>> 🔎 Buscando documentos Text Query: content:definición de documento en el derecho penal AND title:ley AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2 ...
>> 🔎 Buscando documentos Semantic Query: definición de documento en el derecho penal colombiano ...

>> 🧹 Docs filtrados. De 50 pasaron 7 ...
>> ⭐️ Aplicando reorganización semántica ...
>> 🧹 Docs filtrados. De 7 pasaron 7 ...
>> 🔎 Buscando documentos Text Query: content:documento en el derecho penal AND title:jurisprudencia AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2 ...
>> 🔎 Buscando documentos Semantic Query: documento en el derecho penal jurisprudencia colombiana ...

>> 🧹 Docs filtrados. De 50 pasaron 9 ...
>> ⭐️ Aplicando reorganiz

In [103]:
messages = []
run_conversation("¿Cuándo se consuma el hurto?")

🙎‍♂️ User: ¿Cuándo se consuma el hurto?
>> 🤖 Generando respuesta ...
💰 Conversation price: $259.59150000000005 COP (In: 10197, Out: 822, Total token: 11019)
   This message: $25.092000000000002 COP (In: 858, Out: 122, Total token: 980) 
>> 🔎 Buscando documentos Text Query: content:consumación del hurto AND title:ley 599 del 2000 AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2 ...
>> 🔎 Buscando documentos Semantic Query: cuándo se consuma el hurto en la ley 599 del 2000 ...

>> 🧹 Docs filtrados. De 50 pasaron 11 ...
>> ⭐️ Aplicando reorganización semántica ...
>> 🧹 Docs filtrados. De 10 pasaron 10 ...
>> 🔎 Buscando documentos Text Query: content:consumación del hurto AND title:jurisprudencia AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2 ...
>> 🔎 Buscando documentos Semantic Query: cuándo se consuma el hurto según la jurisprudencia colombiana ...

>> 🧹 Docs filtrados. De 50 pasaron 12 ...
>> ⭐️ Aplicando reorganización semán

In [ ]:
messages = []
run_conversation("¿Cuáles son las causales de agravación de la estafa?")

🙎‍♂️ User: ¿Cuáles son las causales de agravación de la estafa?
>> 🤖 Generando respuesta ...
💰 Conversation price: $2001.0460000000003 COP (In: 89155, Out: 2819, Total token: 91974)
   This message: $26.855 COP (In: 1004, Out: 102, Total token: 1106) 
>> 🔎 Buscando documentos: causales de agravación de la estafa artículo 267 ley 599 del 2000 ...
>> 🧹 Docs filtrados. De 50 pasaron 19 ...
>> ⭐️ Aplicando reorganización semántica ...
>> 🧹 Docs filtrados. De 10 pasaron 10 ...
>> 🔎 Buscando documentos: causales de agravación de la estafa ...
>> 🧹 Docs filtrados. De 50 pasaron 40 ...
>> ⭐️ Aplicando reorganización semántica ...
>> 🧹 Docs filtrados. De 10 pasaron 10 ...
>> 🤖 Generando respuesta ...
💰 Conversation price: $2306.4345 COP (In: 103932, Out: 2859, Total token: 106791)
   This message: $305.3885 COP (In: 14777, Out: 40, Total token: 14817) 
>> ⏩️ Buscando los chunks siguientes de 20240813150354universidadexternadoleccionesdederechopenalparteespecialvol12011pdf_chunk498 ...
>> 🤖 Gene

In [19]:
log_file_name = "logs/1 - Queries.log"
category_weight = (
    "category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2"
)
search_semantic_query = (
    "causales de agravación de la estafa artículo 267 ley 599 del 2000"
)
search_text_query = f"content:causales de agravación artículo 267 AND title:ley 599 del 2000 AND {category_weight}"
# Add the category weight here
log(
    f"\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n\n>>>>>>>>>>>>>>>>>>>>>>>>>> 🔎🔎 Text: {search_text_query}\n>>>>>>>>>>>>>>>>>>>>>>>>>> 🔎🔎 Semantic: {search_semantic_query}\n\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n>>>>>>>>>>>>>>>>>>>>>>>>>>"
)
content = search_legal_info_tool(
    text_query=search_text_query, semantic_query=search_semantic_query
)

>> 🔎 Buscando documentos Text Query: content:causales de agravación artículo 267 AND title:ley 599 del 2000 AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2 AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2 ...
>>    Buscando documentos Semantic Query: causales de agravación de la estafa artículo 267 ley 599 del 2000 ...

<iterator object azure.core.paging.ItemPaged at 0x126f7ea80>
{'category': 'Jurisprudencia', 'title': 'CSJ - SP31401(24-06-2009)', 'author': 'Corte Suprema de Justicia', 'keywords': 'Penal', 'namespace': 'arieldocuments', 'external_id': 'ba604551-0e65-4994-8967-cd2cf8ff385f', 'file_path': '20240825204022_csj__sp3140124062009.pdf', 'id': '20240825204022csjsp3140124062009pdf_chunk5', 'content': 'Martha Ligia Rodríguez Salguero /\nCone Arodm aJusti nia\nen documento privado, Título IX, Capítulo Tercero, en concurso\ncon estafa, Título VII, Capítulo Tercero.\nEn ninguna parte de la decisión la Fiscalía imputó de ma

In [35]:
log_file_name = "logs/1 - Queries.log"

In [42]:
log_file_name = "logs/1 - Queries.log"


def ensure_directory(directory_name):
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)


def log(message):
    global log_file_name
    ensure_directory("logs")
    with open(log_file_name, "a") as file:
        file.write(f"{datetime.now()}\n{message}\n")

In [91]:
log_file_name = "logs/1 - Queries.log"
category_weight = (
    "AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2"
)
search_semantic_query = (
    "causales genéricas de atenuación punitiva código penal colombiano"
)
search_text_query = f"content:causales genéricas de atenuación punitiva AND title:ley 599 del 2000 {category_weight}"
# Add the category weight here
log(
    f"\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n\n>>>>>>>>>>>>>>>>>>>>>>>>>> 🔎🔎 Text: {search_text_query}\n>>>>>>>>>>>>>>>>>>>>>>>>>> 🔎🔎 Semantic: {search_semantic_query}\n\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n>>>>>>>>>>>>>>>>>>>>>>>>>>"
)
content = search_legal_info_tool(
    text_query=search_text_query, semantic_query=search_semantic_query
)

>> 🔎 Buscando documentos Text Query: content:causales genéricas de atenuación punitiva AND title:ley 599 del 2000 AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2 AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2 ...
>> 🔎 Buscando documentos Semantic Query: causales genéricas de atenuación punitiva código penal colombiano ...

>> 🧹 Docs filtrados. De 50 pasaron 9 ...
>> ⭐️ Aplicando reorganización semántica ...
>> 🧹 Docs filtrados. De 9 pasaron 9 ...


In [34]:
messages = []
user_input = "¿Cuáles son las causales de agravación de la estafa?"

supporting_info = search_semantic(user_input)

>> 🔎 Buscando documentos de soporte: ¿Cuáles son las causales de agravación de la estafa? ...

<iterator object azure.core.paging.ItemPaged at 0x126f84e90>


KeyboardInterrupt: 

In [49]:
user_input = "¿Cuáles son las causales de agravación de la estafa?"
query_prompt_message = {"role": "system", "content": query_system_template}
user_prompt_message = {
    "role": "user",
    "content": f"Genera uno o varias búsquedas para esta pregunta {user_input}",
}
sources_prompt_message = {
    "role": "assistant",
    "content": f"En una búsqueda inicial, encontré estos extractos. De aquí puedes sacar leyes, artículos, documentos y demás fuentes con las que puedes generar la búsqueda. Solo toma lo que sirva para generar una respuesta al usuario: \n {supporting_info}",
}

messages = [
    query_prompt_message,
    *query_few_shots,
    user_prompt_message,
    sources_prompt_message,
]
print(json.dumps(messages, indent=4))

[
    {
        "role": "system",
        "content": "Eres un experto legal que necesita buscar en una base de conocimiento con cientos de miles de documentos, fuentes confiables y precisas para ayudar a responder las consultas del usuario. He tra\u00eddo algunos extractos de documentos que pueden ser \u00fatiles para que identifiques fuentes que puedes incluir en tu t\u00e9rmino de b\u00fasqueda si ves que son aplicables. La base de conocimientos est\u00e1 alojada en Azure AI Search y bebes generar un query en formato JSON para buscar all\u00ed. \nLos documentos est\u00e1n guardados en fragmentos con los siguientes campos:\n- id: Identificador \u00fanico del fragmento\n- title: T\u00edtulo del documento\n- author: Autor del documento\n- keywords: Tema legal, puede ser: General, Constitucional, Internacional_Publico, Internacional_Privado, Penal, Financiero, entre otros.\n- category: Tipo de documento legal. Las opciones son: Jurisprudencia, Doctrina, Constitucional, Legal, Infralegal 

In [57]:
response = generate_queries()

💰 Conversation price: $190.3425 COP (In: 8010, Out: 425, Total token: 8435)
   This message: $51.721500000000006 COP (In: 2250, Out: 91, Total token: 2341) 


In [58]:
print(response)
print("\n\n")
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-A78nxP8Z7rj4Skghut8lUqLtbaDeM', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"text_query":"content:causales de agravación de la estafa AND title:ley 599 del 2000","semantic_query":"causales de agravación de la estafa en el código penal colombiano","category_weight":{"Constitucional":6,"Legal":5,"Infralegal":4,"Jurisprudencia":3,"Doctrina":2,"Otros_temas_legales":1},"filters":""}', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1726265505, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_b2ffeb16ee', usage=CompletionUsage(completion_tokens=91, prompt_tokens=2250, total_tokens=2341), prompt_filter_results=[{'prom

In [75]:
log_file_name = "logs/1 - Queries.log"
# category_weight = (
#     "AND category:Constitucional^6 Legal^5 Infralegal^4 Jurisprudencia^3 Doctrina^2"
# )
search_semantic_query = (
    "causales de agravación de la estafa en el código penal colombiano"
)
search_text_query = f"content:causales de agravación de la estafa en el código penal colombiano AND title:ley 599 del 2000"
# Add the category weight here
log(
    f"\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n\n>>>>>>>>>>>>>>>>>>>>>>>>>> 🔎🔎 Text: {search_text_query}\n>>>>>>>>>>>>>>>>>>>>>>>>>> 🔎🔎 Semantic: {search_semantic_query}\n\n>>>>>>>>>>>>>>>>>>>>>>>>>>\n>>>>>>>>>>>>>>>>>>>>>>>>>>"
)
content = search_legal_info_tool(
    text_query=search_text_query, semantic_query=search_semantic_query
)

>> 🔎 Buscando documentos Text Query: content:causales de agravación de la estafa en el código penal colombiano AND title:ley 599 del 2000 ...
>>    Buscando documentos Semantic Query: causales de agravación de la estafa en el código penal colombiano ...

<iterator object azure.core.paging.ItemPaged at 0x146278da0>
{'category': 'Jurisprudencia', 'title': 'CSJ - AP983-2021(59054)', 'author': 'Corte Suprema de Justicia', 'keywords': 'Penal', 'namespace': 'arieldocuments', 'external_id': 'f41e1a5f-5b8b-427c-a1f6-aab0345b65a7', 'file_path': '20240825092450_csj__ap983202159054.pdf', 'id': '20240825092450csjap983202159054pdf_chunk12', 'content': 'Según lo expuesto en el escrito de acusación, la cuantía de\nla estafa agravada asciende a $422.262.185 monto que supera\nel equivalente a 150 salarios mínimos legales mensuales\nvigentes para la época en que se dice ocurrieron los hechos\nmateria de juzgamiento.\nDe manera que al no existir asignación especial para el\nconocimiento de dicha conducta